## Pair Programming Limpieza V

Es el momento de ponernos a trabajar con los valores nulos de nuevo 💪🏽. A lo largo de este ejercicio de pair programming vamos a intentar eliminar los valores nulos de nuestras columnas. En la lección hemos aprendido varios métodos de skelarn intentemos aplicarlos todos. Manos a la obra!

1. Es el momento de eliminar los nulos:
- Reemplazad los valores nulos del resto de las columnas categóricas por la moda, usando el método SimpleImputer.   
    💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
if 'limpieza' not in os.getcwd():
    os.chdir('limpieza')

In [2]:
df = pd.read_csv("../datos/bank-additional-full5.csv", index_col = 0)
df.head( )

,age,job,marital,education,housing,loan,contact,duration,campaign,pdays,...,poutcome,"emp,var,rate","cons,price,idx","cons,conf,idx",euribor3m,"nr,employed",y,month_day_week,month,day_week
0,56,housemaid,married,basic 4y,No,No,telephone,261,1,999,...,NONEXISTENT,1.1,93.994,-36.4,4.857,"5191,0",no,"'may', 'mon'",'may','mon'
1,57,services,married,high school,No,No,telephone,149,1,999,...,NONEXISTENT,1.1,93.994,-36.4,4.857,"5191,0",no,"'may', 'mon'",'may','mon'
2,37,services,married,high school,Si,No,telephone,226,1,999,...,NONEXISTENT,1.1,93.994,-36.4,4.857,"5191,0",no,"'may', 'mon'",'may','mon'
3,40,administrator,married,basic 6y,No,No,telephone,151,1,999,...,NONEXISTENT,1.1,93.994,-36.4,4.857,"5191,0",no,"'may', 'mon'",'may','mon'
4,56,services,married,high school,No,Si,telephone,307,1,999,...,NONEXISTENT,1.1,93.994,-36.4,4.857,"5191,0",no,"'may', 'mon'",'may','mon'


In [3]:
# aunque no hay nulos suponemos que los hay para hacer los ejercicios
df.isnull().sum()

age               0
job               0
marital           0
education         0
housing           0
loan              0
contact           0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp,var,rate      0
cons,price,idx    0
cons,conf,idx     0
euribor3m         0
nr,employed       0
y                 0
month_day_week    0
month             0
day_week          0
dtype: int64

In [4]:
df_categoricas = df.select_dtypes(include='object').drop(columns=['education', 'housing', 'loan'], axis=1)
df_categoricas.head()

,job,marital,contact,poutcome,"nr,employed",y,month_day_week,month,day_week
0,housemaid,married,telephone,NONEXISTENT,"5191,0",no,"'may', 'mon'",'may','mon'
1,services,married,telephone,NONEXISTENT,"5191,0",no,"'may', 'mon'",'may','mon'
2,services,married,telephone,NONEXISTENT,"5191,0",no,"'may', 'mon'",'may','mon'
3,administrator,married,telephone,NONEXISTENT,"5191,0",no,"'may', 'mon'",'may','mon'
4,services,married,telephone,NONEXISTENT,"5191,0",no,"'may', 'mon'",'may','mon'


In [5]:
imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
imputer = imputer.fit(df[df_categoricas.columns])

In [6]:
df[df_categoricas.columns] = imputer.transform(df[df_categoricas.columns])

In [7]:
df.isnull().sum()

age               0
job               0
marital           0
education         0
housing           0
loan              0
contact           0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp,var,rate      0
cons,price,idx    0
cons,conf,idx     0
euribor3m         0
nr,employed       0
y                 0
month_day_week    0
month             0
day_week          0
dtype: int64

- Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [8]:
def eliminar_nulos_iterative(dataframe_completo):
    df_numericas = dataframe_completo.select_dtypes(include = np.number)
    imputer = IterativeImputer(n_nearest_features=None, imputation_order='ascending')
    imputer.fit(df_numericas)
    numericas_trans = pd.DataFrame(imputer.transform(df_numericas), columns = df_numericas.columns)
    dataframe_completo.drop(columns = df_numericas.columns, inplace= True)
    dataframe_completo = pd.concat([dataframe_completo, numericas_trans], axis = 1)
    return dataframe_completo

In [9]:
# lo guardamos en otro df2 para poder hacer luego los ejercios sobre la misma columna
df2 = df.copy()
df2 = eliminar_nulos_iterative(df2)  # hay que guardarlo sobre el mismo df2 porque si lo guardamos en otra variable el df2 se queda solo con las categóricas
df2.head()

,job,marital,education,housing,loan,contact,poutcome,"nr,employed",y,month_day_week,...,day_week,age,duration,campaign,pdays,previous,"emp,var,rate","cons,price,idx","cons,conf,idx",euribor3m
0,housemaid,married,basic 4y,No,No,telephone,NONEXISTENT,"5191,0",no,"'may', 'mon'",...,'mon',56.0,261.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857
1,services,married,high school,No,No,telephone,NONEXISTENT,"5191,0",no,"'may', 'mon'",...,'mon',57.0,149.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857
2,services,married,high school,Si,No,telephone,NONEXISTENT,"5191,0",no,"'may', 'mon'",...,'mon',37.0,226.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857
3,administrator,married,basic 6y,No,No,telephone,NONEXISTENT,"5191,0",no,"'may', 'mon'",...,'mon',40.0,151.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857
4,services,married,high school,No,Si,telephone,NONEXISTENT,"5191,0",no,"'may', 'mon'",...,'mon',56.0,307.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857


In [10]:
df2.isnull().sum()

job               0
marital           0
education         0
housing           0
loan              0
contact           0
poutcome          0
nr,employed       0
y                 0
month_day_week    0
month             0
day_week          0
age               0
duration          0
campaign          0
pdays             0
previous          0
emp,var,rate      0
cons,price,idx    0
cons,conf,idx     0
euribor3m         0
dtype: int64

- Reemplazad los valores nulos de la columna age por la media de la edad usando el método KNN Imputer.

In [11]:
def eliminar_nulos_knn(dataframe_completo):
    df_numericas = dataframe_completo.select_dtypes(include = np.number)
    imputer_knn = KNNImputer(n_neighbors=5)
    imputer_knn.fit(df_numericas)
    numericas_trans = pd.DataFrame(imputer_knn.transform(df_numericas), columns = df_numericas.columns)
    dataframe_completo.drop(columns = df_numericas.columns, inplace= True)
    dataframe_completo = pd.concat([dataframe_completo, numericas_trans], axis = 1)
    return dataframe_completo

In [12]:
df3 = df['age'].reset_index()

In [13]:
df3 = eliminar_nulos_knn(df3)
df3.head()

,index,age
0,0.0,56.0
1,1.0,57.0
2,2.0,37.0
3,3.0,40.0
4,4.0,56.0


In [14]:
df3.isnull().sum()

index    0
age      0
dtype: int64

- Reemplazad los valores nulos de la columna age por la media de la edad usando el método Simple Imputer.

In [15]:
imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
imputer = imputer.fit(df[['age']])
df['age'] = imputer.transform(df[['age']])

In [16]:
df.isnull().sum()

age               0
job               0
marital           0
education         0
housing           0
loan              0
contact           0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp,var,rate      0
cons,price,idx    0
cons,conf,idx     0
euribor3m         0
nr,employed       0
y                 0
month_day_week    0
month             0
day_week          0
dtype: int64

- ¿Podríais explicar qué diferencia hay entre estos últimos tres ejercicios?

- El IterativeImputer modifica un dataframe completo. Sustituye cada NaN por el valor que tomaría sobre la línea de regresión lineal que sería la media de todas las iteraciones realizadas.   
- El KNNImputer modifica un dataframe completo. Sustituye cada NaN por la media de esa columna de los valores vecinos que ha encontrado.  
- El SimpleImputer solo modifica una columna o lista de columnas. Sustituye los NaN por la media de esa columna.  

2. Guardad el csv.

In [17]:
df2.to_csv("../datos/bank-additional-full6.csv")